# CSC411 Assignment 3
Let's get it.

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

## P1: Reading Data

In [10]:
import os
root_dir = os.path.join(os.getcwd(), "data")
fake_data = pd.read_fwf(os.path.join(root_dir, "clean_fake.txt"), header=None)
real_data = pd.read_fwf(os.path.join(root_dir, "clean_real.txt"), header=None)

In [11]:
fake_train, fake_test = train_test_split(fake_data, test_size=0.15)
real_train, real_test = train_test_split(real_data, test_size=0.15)

fake_train, fake_validation = train_test_split(fake_train, test_size=0.15/0.85)
real_train, real_validation = train_test_split(real_train, test_size=0.15/0.85)

## P2: Naive Bayes

In [40]:
def generate_counts(data):
    '''
    data is an array of strings
    '''
    counts = {}
    for line in data:
        words = line.split(" ")
        for word in words:
            if word in counts:
                counts[word] += 1.0
            else:
                counts[word] = 1.0
    return counts

In [59]:
fake_counts = generate_counts(fake_train[0])
real_counts = generate_counts(real_train[0])

for word in fake_counts.keys():
    if word not in real_counts:
        real_counts[word] = 0.0
for word in real_counts.keys():
    if word not in fake_counts:
        fake_counts[word] = 0.0

total_counts = {}
for word in real_counts.keys():
    total_counts[word] = real_counts[word] + fake_counts[word]

num_fake = float(len(fake_train))
num_real = float(len(real_train))
num_total = num_fake + num_real

P_real = num_real / num_total
P_fake = num_fake / num_total

fake_freqs = {n: fake_counts[n] / num_fake for n in fake_counts.keys()}      #gives P(x | fake)
real_freqs = {n: real_counts[n] / num_real for n in real_counts.keys()}      #gives P(x | real)
total_freqs = {n: total_counts[n] / num_total for n in total_counts.keys()}  #gives P(x)

In [68]:
import math
math.
def make_prediction(line):
    '''
    line is a string
    '''
    epsilon = 1e-12
    words = line.split(" ")
    P_words_given_real = 1.0
    P_words_given_fake = 1.0
    for word in real_freqs.keys():
      
        if word in words:
            P_words_given_real *= real_freqs[word] + epsilon
            P_words_given_fake *= fake_freqs[word] + epsilon 
        else:
            P_words_given_real *= 1 - real_freqs[word] + epsilon
            P_words_given_fake *= 1 - fake_freqs[word] + epsilon
            
    numerator = P_words_given_real * P_real
    denominator = numerator + P_words_given_fake * P_fake
    #print(numerator)
    #print(denominator)
    return numerator / denominator

In [67]:
print(make_prediction("hillary donald wow united marines sessions"))

1.00000000002


## Part 3

In [44]:
fake_freqs_ = [(word, freq) for word, freq in fake_freqs.items()]
sorted_fake_words = sorted(fake_freqs_, key=lambda pair: pair[1], reverse=True)

In [45]:
sorted_fake_words[:10]

[('trump', 1.0220264317180616),
 ('the', 0.3447136563876652),
 ('to', 0.29845814977973567),
 ('in', 0.183920704845815),
 ('donald', 0.17290748898678415),
 ('a', 0.1685022026431718),
 ('for', 0.1618942731277533),
 ('of', 0.15418502202643172),
 ('and', 0.14757709251101322),
 ('on', 0.12555066079295155)]

## Part 7: Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [ ]:
fake_train_labels = np.zeros((fake_train.shape[0], 1))
real_train_labels = np.ones((real_train.shape[0], 1))
y_train = np.row_stack((fake_train_labels, real_train_labels))
X_train = 

In [20]:
all_depths = []
train_accuracies = []
valid_accuracies = []

In [21]:
max_depths = np.random.randint(low=1, high=100, size=10)
for depth in max_depths:
    print "Max Depth: {}".format(depth)
    all_depths.append(depth)
    
    tree_clf = DecisionTreeClassifier(max_depth=depth)
    tree_clf.fit(fake_train, real_train)
    
    train_accuracy = tree_clf.score(fake_train, real_train)
    valid_accuracy = tree_clf.score(fake_validation, real_validation)
    
    train_accuracies.append(train_accuracy)
    valid_accuracies.append(valid_accuracy)

Max Depth: 93


ValueError: could not convert string to float: comment on rachel maddow declares that a trump presidency could have radical effects on her chosen profession by anonymous